In [ ]:
def multiply_list(lst):
    out = [None] * (2*len(lst))
    out[::2] = lst
    out[1::2] = lst
    return out
def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go
    import plotly.express as px
    import random,pickle
#     import plotly
    
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    user_df=data_df.loc[data_df.user==user_id]
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    random.shuffle(colors)
#     print("I1")
    if len(user_df)>0:
        cats = [ 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
#         user_df["weeks"]=user_df["week"].apply(lambda n: n+(np.random.random_sample()-0.2))

       
        
        user_df.sort_values(['week', 'start_time_of_day'], inplace = True)
        grps = user_df.groupby(['week', 'start_time_of_day'])
        week_new = []
        print("I")
        for grp, df_grp in grps:
            if len(df_grp) == 1:
                week_new.append(grp[0])
            else:
                mid = len(df_grp) // 2
                prev = 0
                prev1= 0
                lower=mid
                upper=len(df_grp)-mid -1 
                for i in range(len(df_grp)):

                    if i > mid:
        #                 bias = np.random.uniform(.01, .25)
                        bias = prev + (0.2/upper) 
                        data = grp[0] + bias
                        week_new.append(data)
                        prev = bias 
                    elif i == mid:
                        week_new.append(grp[0])
                    else:
        #                 bias = np.random.uniform(.01, .25)
                        bias = prev1 + (0.2/lower) 
                        data = grp[0] - bias
                        week_new.append(data)
                        prev1 = bias 
        
        user_df['week_new'] = week_new
        print("I")
#         stressors_list=user_df.Stressor.unique().tolist()
        stressors_list=user_df.Stressor.unique().tolist()
        if len(stressors_list) > len(colors):
            colors=multiply_list(colors)
#         print("I3")
#         print(user_df.Stressor.unique())
        fig = go.Figure()
        button_list=[]
        print(len(colors),len(user_df.Stressor.unique()))
        for i in user_df.Stressor.unique():
            temp_df=user_df.loc[user_df["Stressor"]==i]
#             print(temp_df)
            temp_df=temp_df.sort_values(["week_new"])
            fig.add_trace(go.Scatter(x= temp_df["week_new"], y=temp_df["start_time_of_day"],mode="markers",text=i,customdata=temp_df[["Stressor","week"]],
                                     hovertemplate=
                                     "Week:%{customdata[1]}<br>" + 
                                     "Stressor : %{customdata[0]}<br>"+
                                "Time of day: %{y}<br>" +                                       
#                                  "# of occurences: %{customdata[3]}<br>" +        
                                "<extra></extra>",opacity=1,
#                                      visible='legendonly',
                                     name=str(i),
                                     marker=dict(size=5,color=colors[stressors_list.index(i)])
                                    ))

#         stressors_list=user_df.Stressor.unique().tolist()



        print("I2")
        fig.update_layout( 
            legend=dict(itemsizing= 'constant',itemclick="toggleothers",itemdoubleclick="toggle",orientation="h", y=1.1,xanchor="right",x=1,yanchor="bottom",font_size=10,
                       
                       title_font_family="Times New Roman",title_font_size=12,
        font=dict(
            family="Courier",
            size=10,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2),
            autosize=True,showlegend=True, 
            font=dict(

                size=10,
                color="Black"
            ),
            )
        print("I3")
        fig.update_xaxes(range=[0,int(week_num)+0.5],nticks=int(week_num)+1,dtick=1,title="Week #",)
        fig.update_yaxes( showticklabels=True, visible=True,tickangle = 270,)
        fig.update_yaxes(automargin=True,categoryorder="array",categoryarray=["Morning","Afternoon","Evening","Night"])
        fig.update_layout(legend_title="<b>Daily Stressors</b>",legend_title_side="top",
                    autosize=True,

                    margin=dict(
                        l=20,
                        r=20,
                        b=20,
                        t=40,
                       
                    ),
        title={"text":"",
            "x":0.1,
             "yanchor":"bottom",
        "xanchor":"center",
            
           
        })
        fig.update_layout(hoverlabel=dict(
                                    bgcolor="white",
                                    font_size=16,
                                    font_family="Rockwell",
                                    bordercolor="Black"
                                ),
                          font=dict(
        family="Courier New, monospace",
        size=12,
        color="Black"
    ))

        fig.update_layout(hoverlabel=dict(
                                    bgcolor="white",
                                    font_size=16,
                                    font_family="Rockwell",
                                    bordercolor="Black"
                                )
                            )
       
        data_summary_user=data_summary_user_current["Time of day Trend"]
        user_df_this_week=user_df.loc[user_df["week"] == week_num]
        user_df_prev_week=user_df.loc[user_df["week"] != week_num]
        if len(user_df_this_week) > 0:
            user_df_this_week_group=user_df_this_week.groupby(["start_time_of_day"],as_index=False)["counter"].count().sort_values(["counter"],ascending=False)
            max_time=user_df_this_week_group["counter"].max()
            time_this_week=user_df_this_week_group.loc[user_df_this_week_group["counter"]==max_time]["start_time_of_day"].values.tolist()

            data_summary_user=data_summary_user.replace("_a_", str(" ,".join(time_this_week)))
        else:
            data_summary_user=data_summary_user.replace("_a_", str("No stressors reported this week"))
            
        
        if len(user_df_prev_week) > 0:
            user_df_prev_week_group=user_df_prev_week.groupby(["start_time_of_day"],as_index=False)["counter"].count().sort_values(["counter"],ascending=False)
#             print(user_df_prev_week_group)
            max_time=user_df_prev_week_group["counter"].max()
            
            time_prev_week=user_df_prev_week_group.loc[user_df_prev_week_group["counter"]==max_time]["start_time_of_day"].values.tolist()

            data_summary_user=data_summary_user.replace("_b_", str(" ,".join(time_prev_week)))
        data_summary_user=data_summary_user.replace("week_num", str(week_num))
        
        return "Time of day Trend",fig,data_summary_user
    else:
        return "","",""